In [0]:
CREATE OR REFRESH MATERIALIZED VIEW gold.customer_order_summary 
AS
SELECT 
    c.c_custkey,
    c.c_name,
    COUNT(o.o_orderkey) AS total_orders,
    SUM(o.o_totalprice) AS total_revenue,
    MIN(o.o_orderdate) AS first_order_date,
    MAX(o.o_orderdate) AS last_order_date
FROM silver.customers_silver c
LEFT JOIN silver.orders_silver o
  ON c.c_custkey = o.o_custkey
GROUP BY c.c_custkey, c.c_name;

In [0]:
CREATE OR REFRESH MATERIALIZED VIEW gold.monthly_order_trends
AS
SELECT 
    DATE_TRUNC('month', o.o_orderdate) AS order_month,
    COUNT(o.o_orderkey) AS order_count,
    SUM(o.o_totalprice) AS revenue
FROM silver.orders_silver o
GROUP BY DATE_TRUNC('month', o.o_orderdate)
ORDER BY order_month;

In [0]:
CREATE OR REFRESH STREAMING TABLE exceptions.orders_invalid
AS
SELECT *
FROM stream(bronze.orders)
WHERE o_orderdate IS NULL
;

In [0]:
-- 3. Gold Layer: Dashboard / Expectations summary

CREATE OR REFRESH MATERIALIZED VIEW gold.quality_dashboard_mv
COMMENT "Counts of valid vs invalid orders, customer counts"
AS
SELECT
  (SELECT COUNT(*) FROM silver.orders_silver) AS valid_orders,
  (SELECT COUNT(*) FROM silver.customers_silver) AS total_customers
;

-- 4. Gold Layer: Customer Lifetime Value materialized as view

CREATE OR REFRESH MATERIALIZED VIEW gold.customer_ltv_mv
COMMENT "Lifetime Value summary per customer"
AS
SELECT
  c.c_custkey,
  c.c_name,
  COUNT(o.o_orderkey) AS total_orders,
  SUM(o.o_totalprice) AS total_revenue,
  AVG(o.o_totalprice) AS avg_order_value
FROM silver.customers_silver c
LEFT JOIN silver.orders_silver o ON c.c_custkey = o.o_custkey
GROUP BY c.c_custkey, c.c_name
;